In [10]:
# ============================================================================
# RESIDUAL STREAM ANALYSIS - HTSAT HEADS SPECIALIZATION
# ============================================================================

import torch
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# ML & Analysis
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
from skdim.id import TwoNN

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import pandas as pd

# Custom imports
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50

# Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print(f"✅ Device: {device}")
print(f"✅ All imports successful")

✅ Device: cpu
✅ All imports successful


In [11]:
# ============================================================================
# LOAD DATASET & MODEL
# ============================================================================

# Dataset
root_path = "./data"
dataset = ESC50(root=root_path, download=True)

print(f"📊 Dataset: {len(dataset)} samples, {len(dataset.classes)} classes")
print(f"   Classes: {dataset.classes[:5]}... (+{len(dataset.classes)-5} more)")

# CLAP Model
wrapper = CLAPWrapper(version='2023', use_cuda=torch.cuda.is_available())
clap_model = wrapper.clap
clap_model.eval()

audio_encoder = clap_model.audio_encoder.base.htsat
audio_encoder.eval()

print(f"\n✅ CLAP model loaded and set to eval mode")

Loading audio files


2000it [00:00, 15260.78it/s]

📊 Dataset: 2000 samples, 50 classes
   Classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']... (+45 more)



✅ CLAP model loaded and set to eval mode


Explore the dataset.

In [12]:
dataset.df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [13]:
# Path, label, one-hot encoding of label
dataset[0]

('./data/ESC-50-master/audio/1-100032-A-0.wav',
 'dog',
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [14]:
from IPython.display import Audio

Audio(filename=dataset[0][0])

Preparing the text prompts.

In [15]:
# ============================================================================
# ARCHITECTURE ANALYSIS
# ============================================================================

print("\n" + "="*70)
print("HTSAT ARCHITECTURE")
print("="*70)

print(f"\n📐 Model Structure:")
print(f"   Layers: {audio_encoder.num_layers}")
print(f"   Depths: {audio_encoder.depths}")
print(f"   Heads per layer: {audio_encoder.num_heads}")
print(f"   Embed dim: {audio_encoder.embed_dim}")
print(f"   Window size: {audio_encoder.window_size}")

# Build head info table
head_info = []
total_heads = 0

for layer_idx, (depth, n_heads) in enumerate(zip(audio_encoder.depths, audio_encoder.num_heads)):
    layer_dim = int(audio_encoder.embed_dim * (2 ** layer_idx))
    head_dim = layer_dim // n_heads
    
    for block_idx in range(depth):
        for head_idx in range(n_heads):
            head_info.append({
                'head_id': f"L{layer_idx}_B{block_idx}_H{head_idx}",
                'layer': layer_idx,
                'block': block_idx,
                'head': head_idx,
                'layer_dim': layer_dim,
                'head_dim': head_dim,
                'n_heads': n_heads
            })
            total_heads += 1

head_info_df = pd.DataFrame(head_info)

print(f"\n🎯 Total Attention Heads: {total_heads}")
print(f"\n   Head Dimensions:")
for layer_idx in range(audio_encoder.num_layers):
    layer_dim = int(audio_encoder.embed_dim * (2 ** layer_idx))
    n_heads = audio_encoder.num_heads[layer_idx]
    head_dim = layer_dim // n_heads
    n_heads_layer = audio_encoder.depths[layer_idx] * n_heads
    print(f"   Layer {layer_idx}: {n_heads_layer} heads × {head_dim}D = {layer_dim}D total")

# Display sample
print(f"\n📋 Head Info Sample:")
display(head_info_df.head(184))


HTSAT ARCHITECTURE

📐 Model Structure:
   Layers: 4
   Depths: [2, 2, 6, 2]
   Heads per layer: [4, 8, 16, 32]
   Embed dim: 96
   Window size: 8

🎯 Total Attention Heads: 184

   Head Dimensions:
   Layer 0: 8 heads × 24D = 96D total
   Layer 1: 16 heads × 24D = 192D total
   Layer 2: 96 heads × 24D = 384D total
   Layer 3: 64 heads × 24D = 768D total

📋 Head Info Sample:


,head_id,layer,block,head,layer_dim,head_dim,n_heads
0,L0_B0_H0,0,0,0,96,24,4
1,L0_B0_H1,0,0,1,96,24,4
2,L0_B0_H2,0,0,2,96,24,4
3,L0_B0_H3,0,0,3,96,24,4
4,L0_B1_H0,0,1,0,96,24,4
...,...,...,...,...,...,...,...
179,L3_B1_H27,3,1,27,768,24,32
180,L3_B1_H28,3,1,28,768,24,32
181,L3_B1_H29,3,1,29,768,24,32
182,L3_B1_H30,3,1,30,768,24,32


In [16]:
# ============================================================================
# HEAD EXTRACTOR CLASS (PRE-PROJECTION)
# ============================================================================

class HeadLevelExtractor:
    """
    Extracts individual attention head outputs BEFORE projection.
    Captures (attn @ v) for each head separately.
    """
    def __init__(self, model):
        self.model = model
        self.head_outputs = defaultdict(list)
        self.hooks = []
    
    def _get_head_hook(self, layer_idx, block_idx, num_heads, head_dim):
        def hook(module, input, output):
            x_input = input[0]  # [B_, N, C]
            _, attn = output
            B_, N, C = x_input.shape
            
            # Recompute QKV
            qkv = module.qkv(x_input)
            qkv = qkv.reshape(B_, N, 3, num_heads, head_dim)
            qkv = qkv.permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]
            
            # Extract each head BEFORE projection
            for head_idx in range(num_heads):
                head_output = torch.matmul(
                    attn[:, head_idx, :, :],
                    v[:, head_idx, :, :]
                )
                head_id = f"L{layer_idx}_B{block_idx}_H{head_idx}"
                self.head_outputs[head_id].append(head_output.detach().cpu())
                print("Salvato output " + head_id + ": " + str(head_output.shape))
        
        return hook
    
    def register_hooks(self):
        print("\n📍 Registering hooks...")
        
        for layer_idx, layer in enumerate(self.model.layers):
            num_heads = self.model.num_heads[layer_idx]
            layer_dim = int(self.model.embed_dim * (2 ** layer_idx))
            head_dim = layer_dim // num_heads
            
            for block_idx, block in enumerate(layer.blocks):
                hook = block.attn.register_forward_hook(
                    self._get_head_hook(layer_idx, block_idx, num_heads, head_dim)
                )
                self.hooks.append(hook)
        
        expected_heads = sum([d * h for d, h in zip(self.model.depths, self.model.num_heads)])
        print(f"   ✅ Registered {len(self.hooks)} hooks for {expected_heads} heads")
    
    def remove_hooks(self):
        for hook in self.hooks:
            hook.remove()
        self.hooks = []
    
    def clear_cache(self):
        self.head_outputs.clear()
    
    def get_head_representations(self):
        return self.head_outputs

# Initialize
extractor = HeadLevelExtractor(audio_encoder)
extractor.register_hooks()


📍 Registering hooks...
   ✅ Registered 12 hooks for 184 heads


In [ ]:
# ============================================================================
# EXTRACT HEAD REPRESENTATIONS
# ============================================================================

print("\n" + "="*70)
print("EXTRACTING HEAD REPRESENTATIONS")
print("="*70)

# Config
N_SAMPLES = 50  # Total samples
BATCH_SIZE = 50

samples_per_class = N_SAMPLES // len(dataset.classes)
print(f"\n📝 Configuration:")
print(f"   Total samples: {N_SAMPLES}")
print(f"   Samples per class: {samples_per_class}")
print(f"   Batch size: {BATCH_SIZE}")

# Collect stratified samples
class_samples = defaultdict(list)

for idx in range(len(dataset)):
    audio_path, class_name, one_hot_target = dataset[idx]
    class_idx = torch.argmax(one_hot_target).item()
    
    if len(class_samples[class_idx]) < samples_per_class:
        class_samples[class_idx].append((audio_path, class_idx))
    
    if all(len(samples) >= samples_per_class for samples in class_samples.values()):
        break

# Flatten to single list
sample_list = []
for class_idx, samples in sorted(class_samples.items()):
    sample_list.extend(samples)

print(f"\n✅ Collected {len(sample_list)} samples")
print(f"   Classes covered: {len(class_samples)}/{len(dataset.classes)}")

# Extract with aggregation
head_aggregated = defaultdict(list)
sample_labels = []  # ← USE THIS FOR LABELS!

for batch_start in tqdm(range(0, len(sample_list), BATCH_SIZE), desc="Extracting"):
    batch_samples = sample_list[batch_start:batch_start + BATCH_SIZE]
    extractor.clear_cache()
    
    for audio_path, label in batch_samples:
        audio_tensor = wrapper.load_audio_into_tensor(
            audio_path, wrapper.args.duration, resample=True
        )
        audio_tensor = audio_tensor.reshape(1, -1).to(device)
        
        with torch.no_grad():
            _ = audio_encoder(audio_tensor)
        
        sample_labels.append(label)
    
    # Aggregate: spatial mean over [num_windows, window_size²]
    head_outputs = extractor.get_head_representations()

    for head_id, repr_list in head_outputs.items():
        for repr_tensor in repr_list:
            pooled = repr_tensor.mean(dim=[0, 1])  # [head_dim]
            head_aggregated[head_id].append(pooled.cpu())
    
    del head_outputs
    torch.cuda.empty_cache()

# Convert to tensors
head_outputs_final = {}
for head_id in sorted(head_aggregated.keys()):
    head_outputs_final[head_id] = torch.stack(head_aggregated[head_id])

# Convert labels to numpy
sample_labels = np.array(sample_labels)

# Verify
print(f"\n✅ Extraction Complete:")
print(f"   Heads extracted: {len(head_outputs_final)}")
print(f"   Shape per head: {head_outputs_final[list(head_outputs_final.keys())[0]].shape}")
print(f"   Sample labels: {len(sample_labels)}")
print(f"   Unique classes: {len(set(sample_labels))}")

assert len(sample_labels) == head_outputs_final[list(head_outputs_final.keys())[0]].shape[0], \
    "❌ Labels and representations mismatch!"
print("   ✅ Dimensions verified!")

print("Salvataggio...", end='')
save_dict = {
    "head_outputs_final": head_outputs_final,   # dict of tensors
    "labels": sample_labels               # numpy array
}

#torch.save(save_dict, "head_outputs_final.pt")
print("Saved to head_outputs_final.pt")


EXTRACTING HEAD REPRESENTATIONS

📝 Configuration:
   Total samples: 50
   Samples per class: 1
   Batch size: 50

✅ Collected 1 samples
   Classes covered: 1/50


Extracting:   0%|          | 0/1 [00:00<?, ?it/s]

Shape before:  torch.Size([64, 64, 96])
WindowAttention out-size: torch.Size([64, 64, 96])
Salvato output L0_B0_H0: torch.Size([64, 64, 24])
Salvato output L0_B0_H1: torch.Size([64, 64, 24])
Salvato output L0_B0_H2: torch.Size([64, 64, 24])
Salvato output L0_B0_H3: torch.Size([64, 64, 24])
Shape before:  torch.Size([64, 64, 96])
WindowAttention out-size: torch.Size([64, 64, 96])
Salvato output L0_B1_H0: torch.Size([64, 64, 24])
Salvato output L0_B1_H1: torch.Size([64, 64, 24])
Salvato output L0_B1_H2: torch.Size([64, 64, 24])
Salvato output L0_B1_H3: torch.Size([64, 64, 24])
Shape before:  torch.Size([16, 64, 192])
WindowAttention out-size: torch.Size([16, 64, 192])
Salvato output L1_B0_H0: torch.Size([16, 64, 24])
Salvato output L1_B0_H1: torch.Size([16, 64, 24])
Salvato output L1_B0_H2: torch.Size([16, 64, 24])
Salvato output L1_B0_H3: torch.Size([16, 64, 24])
Salvato output L1_B0_H4: torch.Size([16, 64, 24])
Salvato output L1_B0_H5: torch.Size([16, 64, 24])
Salvato output L1_B0_H6: 